# Data Augmentation Pipeline

Done with albumentations library.

## Imports and Directories

In [1]:
import albumentations as A
import numpy as np
import gcsfs
import cv2
from pathlib import Path

fs = gcsfs.GCSFileSystem()
src = Path("cow-hooves")
dst = Path("augmented-cow-hooves")
img_paths = fs.ls(src)

## Compose Transform

In [2]:
transform = A.Compose([
    A.RandomResizedCrop(size=(1440, 1080), scale=(0.8, 1.0), ratio=(0.75, 1.33), p=1.0),
    A.HorizontalFlip(p=0.5),
    A.ShiftScaleRotate(
        shift_limit=0.15,
        scale_limit=0.2,
        rotate_limit=25,
        border_mode=0,
        p=0.9
    ),
])

/opt/conda/lib/python3.10/site-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)


## Augment Images

In [3]:
for i in range(10):
    for img_path in img_paths:
        # Read image bytes from GCS
        with fs.open(img_path, "rb") as f:
            img_bytes = f.read()

        # Decode bytes to numpy array (OpenCV image)
        image = cv2.imdecode(np.frombuffer(img_bytes, np.uint8), cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Apply your transform
        augmented = transform(image=image)
        aug_img = augmented["image"]

        # Prepare save path
        image_name = f"{i}_{img_path.split('/')[-1]}"
        save_path = f"{dst}/{image_name}"

        # Encode the image to bytes before saving
        success, encoded_img = cv2.imencode(".jpg", cv2.cvtColor(aug_img, cv2.COLOR_RGB2BGR))
        if success:
            with fs.open(save_path, "wb") as f:
                f.write(encoded_img.tobytes())

        print(f"saved: {save_path}")

Saved: augmented-cow-hooves/0_flir_20251012T132854.jpg
Saved: augmented-cow-hooves/0_flir_20251012T133121.jpg
Saved: augmented-cow-hooves/0_flir_20251012T133405.jpg
Saved: augmented-cow-hooves/0_flir_20251012T133511.jpg
Saved: augmented-cow-hooves/0_flir_20251012T134100.jpg
Saved: augmented-cow-hooves/1_flir_20251012T132854.jpg
Saved: augmented-cow-hooves/1_flir_20251012T133121.jpg
Saved: augmented-cow-hooves/1_flir_20251012T133405.jpg
Saved: augmented-cow-hooves/1_flir_20251012T133511.jpg
Saved: augmented-cow-hooves/1_flir_20251012T134100.jpg
Saved: augmented-cow-hooves/2_flir_20251012T132854.jpg
Saved: augmented-cow-hooves/2_flir_20251012T133121.jpg
Saved: augmented-cow-hooves/2_flir_20251012T133405.jpg
Saved: augmented-cow-hooves/2_flir_20251012T133511.jpg
Saved: augmented-cow-hooves/2_flir_20251012T134100.jpg
Saved: augmented-cow-hooves/3_flir_20251012T132854.jpg
Saved: augmented-cow-hooves/3_flir_20251012T133121.jpg
Saved: augmented-cow-hooves/3_flir_20251012T133405.jpg
Saved: aug